In [1]:
# import requests
# from bs4 import BeautifulSoup
# import os
# from concurrent.futures import ThreadPoolExecutor

# url = "https://www.ncei.noaa.gov/data/oceans/argo/gadr/data/indian/2019/01/"
# folder = "./data"
# os.makedirs(folder, exist_ok=True)

# # Get list of .nc files
# r = requests.get(url)
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(r.text, "html.parser")
# files = [node.get("href") for node in soup.find_all("a") if node.get("href").endswith(".nc")]

# def download_file(f):
#     r_file = requests.get(url + f)
#     with open(os.path.join(folder, f), "wb") as file:
#         file.write(r_file.content)
#     print("Downloaded:", f)

# # Download in parallel using 5 threads
# with ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(download_file, files)

# print("Done")


In [8]:
import xarray as xr

file = r"C:\Users\shubh\Downloads\Classroom\Hackathons\SIH 2025\ARGO\Test\data\nodc_D1900975_339.nc"   # replace with one file you downloaded
ds = xr.open_dataset(file)

ds


<xarray.Dataset> Size: 7kB
Dimensions:                       (n_prof: 1, n_param: 3, n_levels: 72,
                                   n_calib: 1, n_history: 6)
Dimensions without coordinates: n_prof, n_param, n_levels, n_calib, n_history
Data variables: (12/65)
    data_type                     object 8B ...
    format_version                object 8B ...
    handbook_version              object 8B ...
    reference_date_time           object 8B ...
    date_creation                 object 8B ...
    date_update                   object 8B ...
    ...                            ...
    history_parameter             (n_history, n_prof) object 48B ...
    history_start_pres            (n_history, n_prof) float32 24B ...
    history_stop_pres             (n_history, n_prof) float32 24B ...
    history_previous_value        (n_history, n_prof) float32 24B ...
    history_qctest                (n_history, n_prof) object 48B ...
    crs                           int32 4B ...
Attributes: (12/50)
    title:                           Argo float vertical profile
    institution:                     AOML
    source:                          Argo float
    history:                         2019-10-20T04:35:26Z boyer convAGDAC.f90...
    references:                      https://www.nodc.noaa.gov/argo/
    comment:                         
    ...                              ...
    time_coverage_end:               2019-01-06T19:10:56Z
    time_coverage_duration:          point
    time_coverage_resolution:        point
    gadr_ConventionVersion:          GADR-3.0
    gadr_program:                    convAGDAC.f90
    gadr_programVersion:             1.2

In [10]:
type(ds)

xarray.core.dataset.Dataset

In [3]:
# import pandas as pd
# import netCDF4
# import os

# data_list = []

# for file_name in os.listdir("./data"):
#     if file_name.endswith(".nc"):
#         ds = netCDF4.Dataset(os.path.join("./data", file_name))
#         # Extract variables
#         float_id = getattr(ds, 'PLATFORM_NUMBER', 'Unknown')
#         cycle = getattr(ds, 'CYCLE_NUMBER', 'Unknown')
#         lat = ds.variables['LATITUDE'][:]
#         lon = ds.variables['LONGITUDE'][:]
#         pres = ds.variables['PRES'][:]
#         temp = ds.variables['TEMP'][:]
#         psal = ds.variables['PSAL'][:]
        
#         # Flatten and store
#         for i in range(len(lat)):
#             data_list.append({
#                 "float_id": float_id,
#                 "cycle": cycle,
#                 "latitude": lat[i],
#                 "longitude": lon[i],
#                 "pressure": pres[i],
#                 "temperature": temp[i],
#                 "salinity": psal[i]
#             })

# df = pd.DataFrame(data_list)
# df.to_parquet("argo_indian_201901.parquet", index=False)


In [15]:
import xarray as xr

ds = xr.open_dataset(r"C:\Users\shubh\Downloads\Classroom\Hackathons\SIH 2025\ARGO\Test\data\nodc_D1900975_340.nc")

print(ds.variables.keys())    # all variables
print(ds.attrs.keys())        # dataset attributes


KeysView(Frozen({'data_type': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Data type
    conventions:  Argo reference table 1, 'format_version': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:  File format version, 'handbook_version': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:  Data handbook version, 'reference_date_time': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of reference for Julian days
    conventions:  YYYYMMDDHHMISS, 'date_creation': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of file creation
    conventions:  YYYYMMDDHHMISS, 'date_update': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of update of this file
    conventions:  YYYYMMDDHHMISS, 'platform_number': <xarray.Variable (n_prof: 1)> Size: 8B
[1 valu

In [16]:
print(ds.data_vars)

Data variables:
    data_type                     object 8B ...
    format_version                object 8B ...
    handbook_version              object 8B ...
    reference_date_time           object 8B ...
    date_creation                 object 8B ...
    date_update                   object 8B ...
    platform_number               (n_prof) object 8B ...
    project_name                  (n_prof) object 8B ...
    pi_name                       (n_prof) object 8B ...
    station_parameters            (n_prof, n_param) object 24B ...
    cycle_number                  (n_prof) float64 8B ...
    direction                     (n_prof) object 8B ...
    data_centre                   (n_prof) object 8B ...
    dc_reference                  (n_prof) object 8B ...
    data_state_indicator          (n_prof) object 8B ...
    data_mode                     (n_prof) object 8B ...
    platform_type                 (n_prof) object 8B ...
    float_serial_no               (n_prof) object 8B ...
 

In [13]:
print(ds.variables.keys())    # all variables


KeysView(Frozen({'data_type': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Data type
    conventions:  Argo reference table 1, 'format_version': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:  File format version, 'handbook_version': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:  Data handbook version, 'reference_date_time': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of reference for Julian days
    conventions:  YYYYMMDDHHMISS, 'date_creation': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of file creation
    conventions:  YYYYMMDDHHMISS, 'date_update': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of update of this file
    conventions:  YYYYMMDDHHMISS, 'platform_number': <xarray.Variable (n_prof: 1)> Size: 8B
[1 valu

In [14]:
print(ds.attrs.keys())        # dataset attributes


dict_keys(['title', 'institution', 'source', 'history', 'references', 'comment', 'user_manual_version', 'Conventions', 'featureType', 'uuid', 'summary', 'file_source', 'keywords', 'keywords_vocabulary', 'creator_name', 'creator_url', 'creator_email', 'id', 'naming_authority', 'standard_name_vocabulary', 'Metadata_Conventions', 'publisher_name', 'publisher_url', 'publisher_email', 'date_created', 'date_modified', 'date_issued', 'acknowledgment', 'license', 'cdm_data_type', 'geospatial_lat_min', 'geospatial_lat_max', 'geospatial_lon_min', 'geospatial_lon_max', 'geospatial_vertical_min', 'geospatial_vertical_max', 'geospatial_lat_units', 'geospatial_lat_resolution', 'geospatial_lon_units', 'geospatial_lon_resolution', 'geospatial_vertical_units', 'geospatial_vertical_resolution', 'geospatial_vertical_positive', 'time_coverage_start', 'time_coverage_end', 'time_coverage_duration', 'time_coverage_resolution', 'gadr_ConventionVersion', 'gadr_program', 'gadr_programVersion'])


In [17]:
# import xarray as xr
# import pandas as pd
# import glob

# files = glob.glob("./data/*.nc")
# all_data = []

# for f in files:
#     ds = xr.open_dataset(f)
    
#     float_id = ds['platform_number'].values[0]
#     cycle = int(ds['cycle_number'].values[0])
#     lat = float(ds['latitude'].values[0])
#     lon = float(ds['longitude'].values[0])
#     timestamp = pd.to_datetime(ds['juld'].values[0])
    
#     pres = ds['pres'].values[0]          # shape: (n_levels,)
#     temp = ds['temp'].values[0]
#     psal = ds['psal'].values[0]
    
#     # Optional QC flags
#     pres_qc = ds['pres_qc'].values[0]
#     temp_qc = ds['temp_qc'].values[0]
#     psal_qc = ds['psal_qc'].values[0]
    
#     n_levels = pres.shape[0]
    
#     for i in range(n_levels):
#         all_data.append({
#             "float_id": float_id,
#             "cycle": cycle,
#             "latitude": lat,
#             "longitude": lon,
#             "timestamp": timestamp,
#             "pressure": pres[i],
#             "temperature": temp[i],
#             "salinity": psal[i],
#             "pressure_qc": pres_qc[i],
#             "temperature_qc": temp_qc[i],
#             "salinity_qc": psal_qc[i]
#         })

# df = pd.DataFrame(all_data)
# df.to_parquet("argo_indian_201901.parquet", index=False)


In [ ]:
import xarray as xr
import pandas as pd
import glob
import os

input_folder = r"C:\Users\shubh\Downloads\Classroom\Hackathons\SIH 2025\ARGO\Test\data"
output_folder = r"C:\Users\shubh\Downloads\Classroom\Hackathons\SIH 2025\ARGO\Test\processed"
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "argo_indian_2019_01.parquet")

def process_nc(file_path):
    ds = xr.open_dataset(file_path)
    dfs = []

    n_levels = ds.sizes['n_levels']
    n_prof = ds.sizes['n_prof']

    for p in range(n_prof):
        # Get profile-level info
        lat = float(ds['latitude'].values[p])
        lon = float(ds['longitude'].values[p])
        juld = pd.Timestamp(ds['juld'].values[p])
        float_id = str(ds['platform_number'].values[p])
        cycle = int(ds['cycle_number'].values[p])
        
        # Profile data per level
        pres = ds['pres'].values[p, :].flatten()
        temp = ds['temp'].values[p, :].flatten()
        psal = ds['psal'].values[p, :].flatten()

        df = pd.DataFrame({
            "float_id": [float_id]*n_levels,
            "cycle": [cycle]*n_levels,
            "latitude": [lat]*n_levels,
            "longitude": [lon]*n_levels,
            "time": [juld]*n_levels,
            "pressure": pres,
            "temperature": temp,
            "salinity": psal
        })
        dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

# ----------------------------
# Process all files
# ----------------------------
all_files = glob.glob(os.path.join(input_folder, "*.nc"))
all_dfs = []

print(f"Found {len(all_files)} NetCDF files. Processing...")

for f in all_files:
    try:
        df = process_nc(f)
        all_dfs.append(df)
        print(f"Processed: {os.path.basename(f)}")
    except Exception as e:
        print(f"Error processing {f}: {e}")

if all_dfs:
    argo_df = pd.concat(all_dfs, ignore_index=True)
    argo_df.to_parquet(output_file, index=False)
    print(f"Done! Combined data saved to: {output_file}")
else:
    print("No data processed. Check your files.")


In [23]:
# !pip install argopy
!pip install --upgrade argopy xarray netCDF4 scipy


  Using cached argopy-1.3.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached xarray-2025.6.1-py3-none-any.whl.metadata (12 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
INFO: pip is looking at multiple versions of argopy to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
    --------------------------------------- 0.5/41.3 MB 2.1 MB/s eta 0:00:20
    --------------------------------------- 0.5/41.3 MB 2.1 MB/s eta 0:00:20
    --------------------------------------- 0.5/41.3 MB 2.1 MB/s eta 0:00:20
    --------------------------------------- 0.5/41.3 MB 2.1 MB/s eta 0:00:20
    --------------------------------------- 0.8/41.3 MB 549.6 kB/s eta 0:01:14
   - -------------------------------------- 1.0/41.3 MB 708.5 kB/s eta 0:00:57
   - -------------------------------------- 1.0/41.3

  You can safely remove it manually.
  You can safely remove it manually.


In [24]:
from argopy import DataFetcher as Argo

# Choose backend: 'erddap' (default) or 'gdac'
argo = Argo(src='erddap')

# Fetch all profiles from one float
ds = argo.float(1900975).to_xarray()
print(ds)


FSTimeoutError: 

In [25]:
from argopy import DataFetcher as Argo

argo = Argo(src="erddap")

# Fetch float data directly into a pandas DataFrame
df = argo.float(1900975).to_dataframe()
print(df.head())


FSTimeoutError: 